## Population script

In [5]:
import os

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'kikify_django.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django

django.setup()

from kikify.models import Artist, Album, Song
from django.utils import timezone
import stagger
from stagger.id3 import *       # contains ID3 frame types

Getting all mp3 files in directory

In [6]:
def mp3gen(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            if os.path.splitext(filename)[1] == ".mp3":
                yield os.path.join(root, filename)

#### Iterate files:

In [7]:
PATH = 'C:\\Users\\Kristijan\\Desktop\\Muzika'

Util function

In [8]:
def parseTag(tag):
    artist_tag = None
    try:
        artist_tag = tag.artist
        if not artist_tag:
            artist_tag = 'Unknown artist'
    except:
        artist_tag = 'Unknown artist'

    album_tag = None
    try:
        album_tag = tag.album
        if not album_tag:
            album_tag = 'Unknown album'
    except:
        album_tag = 'Unknown album'

    song_tag = None
    try:
        song_tag = tag.title
        if not song_tag:
            song_tag = 'Unknown song'
    except:
        song_tag = 'Unknown song'

    year = None
    try:
        year = int(tag.date.split(sep='-')[0])
    except ValueError:
        year = 2020

    picture = None
    try:
        picture = tag.picture
    except:
        picture = open('../kikify/static/no-album-art.png').read()

    return {
        'song' : song_tag,
        'album': album_tag,
        'artist': artist_tag,
        'year': year,
        'picture': picture
    }

In [9]:
for mp3file in mp3gen(PATH):

    tag = stagger.read_tag(mp3file)
    if not tag: continue


    parsedTag = parseTag(tag=tag)

    # Creating artist
    artists = list(Artist.objects.filter(name=parsedTag['artist']))
    artist = None
    if len(artists)==0:
        artist = Artist.objects.create(name=parsedTag['artist'])
    else:
        artist = artists[0]

    # Creating album
    albums = list(Album.objects.filter(name=parsedTag['album'],
                  year_of_production=parsedTag['year'],artist=artist))
    album = None
    if len(albums)==0:
        album = Album(name=parsedTag['album'],
                  year_of_production=parsedTag['year'],
                  picture=parsedTag['picture'])
        album.artist.set(album)
    else:
        album = albums[0]

    # Creating song
    song_in_bytes = open(file=mp3file, mode='rb').read()
    songs = list(Album.objects.filter(name=parsedTag['title'],
                      year_of_production=parsedTag['year'],
                      album=album))
    song = None
    if len(songs)==0:
        song = Song.objects.create(name=parsedTag['title'],
                    year_of_production=parsedTag['year'],
                    album=album,
                    song_in_bytes=song_in_bytes)
    else:
        song = songs[0]

    print(f'Song {song.name} has been saved successfully.')




ValueError: "<Album: Unknown album>" needs to have a value for field "id" before this many-to-many relationship can be used.